In [12]:
# 진짜진짜 나혼자 도전이다 ㅋ.ㅋ 망할지라도 ㅋㅋㅋ
import numpy as np
import pandas as pd

In [13]:
df = pd.read_csv('../static/data/IMDB/labeledTrainData.tsv', header=0, sep='\t', quoting=3)
df.head(3)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [15]:
df.review[0][:1000] #첫번째글. 1000번캐릭터까지확인
#<br /><br /> 태그가 있다

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

In [16]:
# <br />태그는 공백으로 변환
df['review'] = df.review.str.replace('<br />', ' ')

In [17]:
# 영어 이외의 문자는 공백으로 변환(제거)
import re

df['review'] = df.review.apply(lambda x: re.sub('[^a-zA-Z]', ' ', x)) # X에 대해서 [^a-zA-Z]아닌놈^들은 공백으로 바꿔라

In [18]:
from sklearn.model_selection import train_test_split

feature_df = df.drop(['id', 'sentiment'], axis=1, inplace=False)
X_train, X_test, y_train, y_test = train_test_split(
    feature_df, df.sentiment, test_size=0.3, random_state=156 #feature_df데이터프레임을 줬더니, 아래서 컬럼지점해줘야하더라.. /y값 시리즈 df.sentiment 로 줌
)
X_train.shape, X_test.shape

((17500, 1), (7500, 1))

In [19]:
X_train.head()

,review
3724,This version moved a little slow for my taste...
23599,I really enjoyed this film because I have a t...
11331,Saw this in the theater in and fell out o...
15745,Recently I was looking for the newly issued W...
845,Escaping the life of being pimped by her fath...


In [20]:
y_train.head()

3724     0
23599    1
11331    1
15745    1
845      1
Name: sentiment, dtype: int64

In [42]:
df_test =pd.DataFrame(data=X_test, columns=X_test.columns)
df_test['target'] = y_test

In [43]:
df_test

,review,target
1692,My girlfriend and I were stunned by how bad t...,0
13392,What do you expect when there is no script to...,0
21063,This is a German film from that is somet...,0
10335,Richard Tyler is a little boy who is scared o...,0
16847,I run a group to stop comedian exploitation a...,0
...,...,...
14848,If you like to comment on films where the scr...,0
8450,First let me say that Notorious is an absolu...,1
8221,Realistic movie sure except for the fact that...,0
10638,I will spend a few days dedicated to Ron Howa...,0


In [44]:
#웹에서 쓸 테스트이터저장.. 그런데 전체아니고 테스트데이터만 저장하면 되나?
#인덱스/리뷰/타겟 (문제,정답까지 같이 만든느거 맞나?)
df_test.to_csv('../static/data/IMDB/test.csv', index=False)

In [21]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [22]:
#CountVectorizer의 경우,
count_vect = CountVectorizer(stop_words='english', ngram_range=(1,2))
count_vect.fit(X_train.review) #df다보니 컬럼지정해줘야함 .review
X_train_count = count_vect.transform(X_train.review) # 꼭 두단계를 거쳐야 train, test같은 인덱스를 가짐
X_test_count = count_vect.transform(X_test.review)

In [24]:
X_train_count.shape, X_test_count.shape

((17500, 1296566), (7500, 1296566))

In [25]:
lr_clf = LogisticRegression(C=10)
lr_clf.fit(X_train_count, y_train)
pred = lr_clf.predict(X_test_count)
accuracy_score(y_test, pred) #결과값 정확도0.886 88%

0.886

In [26]:
#TfidfVectorizer
tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
tfidf_vect.fit(X_train.review)
X_train_tfidf = tfidf_vect.transform(X_train.review) # 꼭 두단계를 거쳐야 train, test같은 인덱스를 가짐
X_test_tfidf = tfidf_vect.transform(X_test.review)

In [27]:
#모델저장
import joblib
joblib.dump(tfidf_vect, '../static/model/imdb_vect.pkl') #피클데이터라 뒤에 .pkl붙여주는게 관례
joblib.dump(lr_clf, '../static/model/imdb_lr.pkl')

['../static/model/imdb_lr.pkl']

In [28]:
#모델불러오기
new_vect = joblib.load('../static/model/imdb_vect.pkl')
new_lr = joblib.load('../static/model/imdb_lr.pkl')

In [29]:
#왜 count만하고 tfidf는 안했지?
lr_clf = LogisticRegression(C=10)
lr_clf.fit(X_train_tfidf, y_train)
pred = lr_clf.predict(X_test_tfidf)
accuracy_score(y_test, pred) #결과값0.8936

0.8936

In [23]:
#이건뭐지? 갑툭
new_X_test = new_vect.transform(X_test.review)

In [24]:
pred = new_lr.predict(new_X_test)
accuracy_score(y_test, pred)

0.8648

In [30]:
#파이프라인 학습/예측 평가 (coun + lr)
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('count_vect', CountVectorizer(stop_words='english', ngram_range=(1,2))),
    ('lr_clf', LogisticRegression(C=5))
])

pipeline.fit(X_train.review, y_train)
pred = pipeline.predict(X_test.review)
acc = accuracy_score(y_test, pred)
print(f'Count Vectorizer + Logistic Regression 정확도: {acc:.4f}')
#10:03~ 결과값 0.8860........................................................파람지정안헀으니기본으로되는건가 그리드서치cv해야하나?

Count Vectorizer + Logistic Regression 정확도: 0.8865


In [31]:
joblib.dump(pipeline, '../static/model/IMDB_count_lr.pkl')

['../static/model/IMDB_count_lr.pkl']

In [31]:
#..................이거왜.......................여기부터
new_pipe = joblib.load('../static/model/pipeline.pkl')

In [32]:
pred = new_pipe.predict(X_test.review)
acc = accuracy_score(y_test, pred)
print(f'Count Vectorizer + Logistic Regression 정확도: {acc:.4f}')
#저장한 모델도 위와같은 결과 정확도 0.8860

Count Vectorizer + Logistic Regression 정확도: 0.8865


In [33]:
new_vect.get_params().keys()

dict_keys(['analyzer', 'binary', 'decode_error', 'dtype', 'encoding', 'input', 'lowercase', 'max_df', 'max_features', 'min_df', 'ngram_range', 'norm', 'preprocessor', 'smooth_idf', 'stop_words', 'strip_accents', 'sublinear_tf', 'token_pattern', 'tokenizer', 'use_idf', 'vocabulary'])

In [34]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('new_vect', CountVectorizer(stop_words='english', ngram_range=(1,2))),
    ('new_lr', LogisticRegression(C=5))
])
params = {
    'new_vect__max_df': [100], #300 500
    'new_lr__C': [1]  #5, 10
}

grid_pipe = GridSearchCV(pipeline, param_grid=params, cv=3, scoring='accuracy', verbose=1) # n_jobs=-1
grid_pipe.fit(X_train.review, y_train)
print(grid_pipe.best_params_, grid_pipe.best_score_)
#..................이거왜.......................................여기까지

NameError: name 'GridSearchCV' is not defined

In [32]:
#파이프라인 학습/예측 평가 (tfidf + lr)
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2))),
    ('lr_clf', LogisticRegression(C=5))
])

pipeline.fit(X_train.review, y_train)
pred = pipeline.predict(X_test.review)
acc = accuracy_score(y_test, pred)
print(f'Tfidf Vectorizer + Logistic Regression 정확도: {acc:.4f}')

Tfidf Vectorizer + Logistic Regression 정확도: 0.8916


In [33]:
joblib.dump(pipeline, '../static/model/IMDB_tfidf_lr.pkl')

['../static/model/IMDB_tfidf_lr.pkl']

In [65]:
#파이프라인 학습/예측 평가 (tfidf + svc)
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2))),
    ('svc', SVC())
])

pipeline.fit(X_train.review, y_train)
pred = pipeline.predict(X_test.review)
acc = accuracy_score(y_test, pred)
print(f'Tfidf Vectorizer + SVC정확도: {acc:.4f}')
#11:49-12:15 (25분소요) #정확도0.8860

Tfidf Vectorizer + SVC정확도: 0.8860


In [66]:
joblib.dump(pipeline, '../static/model/IMDB_tfidf_sv.pkl')

['../static/model/IMDB_tfidf_sv.pkl']

In [67]:
#TEST
# 도즈언~~~~
count_lr = joblib.load('../static/model/IMDB_count_lr.pkl')
tfidf_lr = joblib.load('../static/model/IMDB_tfidf_lr.pkl')
tfidf_sv = joblib.load('../static/model/IMDB_tfidf_sv.pkl')

In [48]:
df = pd.read_csv('../static/data/IMDB/test.csv')

In [52]:
index=7496

In [49]:
df #7499개있음

,review,target
0,My girlfriend and I were stunned by how bad t...,0
1,What do you expect when there is no script to...,0
2,This is a German film from that is somet...,0
3,Richard Tyler is a little boy who is scared o...,0
4,I run a group to stop comedian exploitation a...,0
...,...,...
7495,If you like to comment on films where the scr...,0
7496,First let me say that Notorious is an absolu...,1
7497,Realistic movie sure except for the fact that...,0
7498,I will spend a few days dedicated to Ron Howa...,0


In [53]:
label = df.target[index] 
label

1

In [54]:
df.review[7495]
#예를들어 7495번 하면 스트링이 나온다;;(스칼라) 

' If you like to comment on films where the script arrive halfway the movie then this is the one  A setting and acting as in a Porn movie but nothing is happening only some groping and touching of the third kind  Which actually becomes very boring after    minutes of touchy feely but no action  A few of the actors I ve seen in real x rated movies and there their acting then was a lot better  All the special effects are done by the great   Rondo   Whom performs all the magic whit his mind  A cult movie is written on the box  Does that mean that this film is not to be watched at all     Get drunk with some friends and watch this movie on new years eve ore thanks giving  '

In [56]:
# test_data 1차원으로 만드는 방법 1. [리스트]
test_review = []
test_review.append(df.review[index]) # 스트링을 통채로 어펜드해야함
test_review

[' First  let me say that Notorious is an absolutely charming film  very lovingly rendered of its time and subject s   Gretchen Mol is utterly  painfully convincing  the very soul of the contradictions smoothly reified by Ms  Page herself  Irving and Paula Klaw are richly drawn as the working class stiffs they were  having met Paula at Movie Star News in      I can say that Lili Taylor s performance is unimpeachable   and Jared Harris as John Willie  Coutts  is an adoringly debauched genius  Anyone with an interest in the recorded history of American attitudes toward sexuality must see this movie  in a theater preferably  where votes made with dollars count more   Second  I will allow that I am a producer of material similar to that for which the Klaws would become famous  which is no way affects my estimation of Ms  Harron s work as the splendid piece that it is  but does condition my view of Notorious as an act of political resistance of the first order  Ms  Harron has crafted a work

In [57]:
# test_data 1차원으로 만드는 방법 1. [리스트]
test_review = df.iloc[index:index+1, 0].values 
test_review

array([' First  let me say that Notorious is an absolutely charming film  very lovingly rendered of its time and subject s   Gretchen Mol is utterly  painfully convincing  the very soul of the contradictions smoothly reified by Ms  Page herself  Irving and Paula Klaw are richly drawn as the working class stiffs they were  having met Paula at Movie Star News in      I can say that Lili Taylor s performance is unimpeachable   and Jared Harris as John Willie  Coutts  is an adoringly debauched genius  Anyone with an interest in the recorded history of American attitudes toward sexuality must see this movie  in a theater preferably  where votes made with dollars count more   Second  I will allow that I am a producer of material similar to that for which the Klaws would become famous  which is no way affects my estimation of Ms  Harron s work as the splendid piece that it is  but does condition my view of Notorious as an act of political resistance of the first order  Ms  Harron has crafted 

In [68]:
#로딩한모델 count_lr / tfidf_lr / tfsvc
count_pred = count_lr.predict(test_review)
tfidf_pred = tfidf_lr.predict(test_review)
tfsvc_pred = tfidf_sv.predict(test_review)

In [69]:
print(count_pred[0], label)
print(tfidf_pred[0], label)
print(tfsvc_pred[0], label)

1 1
1 1
1 1
